In [21]:
!pip install scikit-build
!pip install pyikarus  --no-build-isolation --verbose --upgrade


Using pip 23.0.1 from /dune/dune-common/build-cmake/dune-env/lib/python3.10/site-packages/pip (python 3.10)
  Using cached pyikarus-0.3.0.dev202310000043.tar.gz (572 kB)
  Preparing metadata (pyproject.toml) ...   Running command Preparing metadata (pyproject.toml)
  Note: dune-common will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-grid will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-geometry will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  Note: dune-istl will get version requirement (<= 2.9.0) in pyproject.toml as stated in Python-Requires.
  running dist_info
  creating /tmp/pip-modern-metadata-fgtivp1d/pyikarus.egg-info
  writing /tmp/pip-modern-metadata-fgtivp1d/pyikarus.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-modern-metadata-fgtivp1d/pyikarus.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-modern-metad

### Import all packages

In [22]:
import matplotlib
import dune.grid
import dune.functions
import pyikarus as iks
import pyikarus.finite_elements
import pyikarus.utils
import pyikarus.assembler
import pyikarus.dirichletValues
import numpy as np
import scipy as sp
from dune.vtk import vtkUnstructuredGridWriter, vtkWriter, RangeTypes, FieldInfo


ImportError: generic_type: type "ScalarAffordances" is already registered!

### Create grid

In [23]:
lowerLeft = [-1,-1]
upperRight = [1,1]
elements = [3,3]

grid = dune.grid.structuredGrid(lowerLeft,upperRight,elements)
grid.hierarchicalGrid.globalRefine(1)
grid.plot()

DUNE-INFO: Generating dune-py module in /dune/dune-common/build-cmake/dune-env/.cache/dune-py
DUNE-INFO: Compiling yasp coordinates dim=2 ctype=double (new)
DUNE-INFO: Compiling HierarchicalGrid (new)
DUNE-INFO: Compiling ReferenceElements (new)


### Add Lagrangian basis

In [ ]:
basisLagrange1 = dune.functions.defaultGlobalBasis(grid, dune.functions.Power(dune.functions.Lagrange(order=1),2))
print('We have {} dofs.'.format(len(basisLagrange1)))
print('We have {} vertices.'.format(grid.size(2)))
print('We have {} elements.'.format(grid.size(0)))

We have 98 dofs.
We have 49 vertices.
We have 36 elements.


### Init load factor and displacement vector

In [ ]:
d = np.zeros(len(basisLagrange1))
lambdaLoad = iks.ValueWrapper(3.0)

### Create finite element requirements

In [ ]:
req= iks.FErequirements()
req.addAffordance(iks.ScalarAffordances.mechanicalPotentialEnergy)
req.addAffordance(iks.VectorAffordances.forces)
req.addAffordance(iks.MatrixAffordances.stiffness)

req.insertParameter(iks.FEParameter.loadfactor,lambdaLoad)
req.insertGlobalSolution(iks.FESolutions.displacement,d)

AttributeError: module 'pyikarus' has no attribute 'VectorAffordances'

### Define volume load and boundary loads

In [ ]:
def volumeLoad(x,lambdaVal) :
    return np.array([lambdaVal*x[0]*2, 2*lambdaVal*x[1]*0])

def neumannLoad(x,lambdaVal) :
    return np.array([lambdaVal*0, lambdaVal])

neumannVertices = np.zeros(grid.size(2)*2, dtype=bool)
basisLagrange1.interpolate(neumannVertices, lambda x :  True  if x[1]>0.9 else False)

boundaryPatch = iks.utils.boundaryPatch(grid,neumannVertices)

### Create vector of finite elements

In [ ]:
fes = []
for e in grid.elements:
    fes.append(iks.finite_elements.linearElasticElement(basisLagrange1,e,1000,0.2,volumeLoad,boundaryPatch,neumannLoad))

### print forces and stiffness of first element

In [ ]:
forces = np.zeros(8)
stiffness = np.zeros((8,8))
fes[0].calculateVector(req,forces)
fes[0].calculateMatrix(req,stiffness)
np.set_printoptions(precision=3)
print('Forces:\n {}'.format(forces))
print('Stiffness:\n {}'.format(stiffness))

### Create Dirichlet boundary conditions

In [ ]:
dirichletValues = iks.dirichletValues.dirichletValues(basisLagrange1) 

def fixLeftHandEdge(vec,localIndex,localView,intersection):
    if (intersection.geometry.center[1]<-0.9):
        vec[localView.index(localIndex)]= True

dirichletValues.fixBoundaryDOFsUsingLocalViewAndIntersection(fixLeftHandEdge)

### Create assembler

In [ ]:
assembler = iks.assembler.sparseFlatAssembler(fes,dirichletValues)
assemblerDense = iks.assembler.denseFlatAssembler(fes,dirichletValues)

Msparse = assembler.getMatrix(req)
forces = assembler.getVector(req)
print(Msparse)
print(forces)

### Solve for displacements and write to paraview

In [ ]:
d = sp.sparse.linalg.spsolve(Msparse,-forces)
fx = basisLagrange1.asFunction(x)

writer = vtkWriter( grid, "nameTest",  pointData   = {( "displacement",(0,1)):fx})

In [ ]:
!apt update && apt install python3-paraview  -y

Hit:1 http://deb.debian.org/debian bookworm InRelease
Hit:2 http://deb.debian.org/debian bookworm-updates InRelease
Hit:3 http://deb.debian.org/debian-security bookworm-security InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
6 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  distro-info-data gir1.2-glib-2.0 gir1.2-packagekitglib-1.0 gpg gpgconf
  iso-codes libappstream4 libassuan0 libcap2-bin libduktape207
  libgirepository-1.0-1 libglib2.0-bin libgstreamer1.0-0
  libpackagekit-glib2-18 libpolkit-agent-1-0 libpolkit-gobject-1-0
  libstemmer0d libxmlb2 lsb-release packagekit polkitd python-apt-common
  python3-apt python3-blinker python3-dbus python3-distro python3-gi
  python3-httplib2 python3-jwt python3-lazr.restfulclient python3-l

In [ ]:
!apt install python3-paraview

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-paraview is already the newest version (5.11.0+dfsg-1+b1).
The following packages were automatically installed and are no longer required:
  distro-info-data gir1.2-glib-2.0 gir1.2-packagekitglib-1.0 gpg gpgconf
  iso-codes libappstream4 libassuan0 libcap2-bin libduktape207
  libgirepository-1.0-1 libglib2.0-bin libgstreamer1.0-0
  libpackagekit-glib2-18 libpolkit-agent-1-0 libpolkit-gobject-1-0
  libstemmer0d libxmlb2 lsb-release packagekit polkitd python-apt-common
  python3-apt python3-blinker python3-dbus python3-distro python3-gi
  python3-httplib2 python3-jwt python3-lazr.restfulclient python3-lazr.uri
  python3-oauthlib python3-software-properties python3-wadllib sgml-base
  xml-core
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [ ]:
!pip install pv-visualizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.3/396.3 kB 7.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.8/502.8 kB 10.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.8/179.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.9/708.9 kB 10.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 3.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.9/453.9 kB 10.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.3/221.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.4/339.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB

In [ ]:
#import sys

#print(sys.path)
#from paraview.simple import *

['/workspaces/visualStudioCodeAndIkarus', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/dune/dune-common/build-cmake/dune-env/lib/python3.10/site-packages', '__editable__.dune_common-2.9.0.finder.__path_hook__', '/dune/dune-geometry/build-cmake/python', '/dune/dune-localfunctions/build-cmake/python', '/dune/dune-grid/build-cmake/python', '/dune/dune-istl/build-cmake/python', '/dune/dune-functions/build-cmake/python', '/dune/dune-alugrid/build-cmake/python', '/dune/dune-foamgrid/build-cmake/python', '/dune/dune-vtk/build-cmake/python']


ModuleNotFoundError: No module named 'paraview'